Klasu **Graph** treba koristiti za predstavljanje i izgradnju filogenetskog stabla.

In [7]:
class Graph:
    #konstruktorska funkcija (metod)
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    #metod koji vraca stringovsku reprezentaciju grafa    
    def __str__(self):
        return f'{self.adjacency_list}'
    
    #metod koji vraca listu suseda cvora v u grafu
    def get_neighbors(self, v):
        return self.adjacency_list[v]
    
    #metod koji dodaje u graf suseda cvora node sa oznakom neighbor na udaljenosti distance
    def add_neighbor(self, node, neighbor, distance):
        self.adjacency_list[node].append((neighbor, distance))
        
        if neighbor not in self.adjacency_list:
            self.adjacency_list[neighbor] = []
            
        self.adjacency_list[neighbor].append((node, distance))
        
    #metod koji uklanja iz grafa suseda cvora node sa oznakom neighbor na udaljenosti distance    
    def remove_neighbor(self, node, neighbor, distance):
        self.adjacency_list[node].remove((neighbor, distance))
        self.adjacency_list[neighbor].remove((node, distance))    
       
    #metod koji vraca duzinu grane izmedju susednih cvorova node_i i node_j
    def distance_between_nodes(self, node_i, node_j):
        node_i_neighbors = self.get_neighbors(node_i)
    
        for (w, weight) in node_i_neighbors:
            if w == node_j:
                return weight
        
        return None  
    
    #metod koji dodaje novi cvor izmedju cvorova node_i i node_j 
    #koji se nalazi na rastojanju distance_i od cvora node_i
    def add_new_node_between_nodes(self, node_i, node_j, distance_i):
        new_node = '{}+{}'.format(node_i, node_j)
    
        distance_ij = self.distance_between_nodes(node_i, node_j)
        distance_j = distance_ij - distance_i
    
        self.remove_neighbor(node_i, node_j, distance_ij)
        self.add_neighbor(node_i, new_node, distance_i)
        self.add_neighbor(node_j, new_node, distance_j)
    
        return new_node
    
    #metod koji pronalazi put izmedju cvorova source i destination u grafu (DFS obilazak)
    def find_path(self, source, destination):     
        stack = [source]
        visited = set([source])
    
        while len(stack) > 0:
            v = stack[-1]
        
            if v == destination:
                return stack
        
            has_neighbors = False
        
            for (w, weight) in self.get_neighbors(v):
                if w not in visited:
                    has_neighbors = True
                    stack.append(w)
                    visited.add(w)
                    break
        
            if not has_neighbors:
                stack.pop()
            
        print('Path not found')
        return []
    
    #metod koji dodaje novi unutrasnji cvor ili vraca vec postojeci unutrasnji cvor na putu izmedju 
    #cvorova source i distance koji se nalazi na rastojanju distance_from_source od cvora source
    def add_node_on_path(self, source, destination, distance_from_source):
        path = self.find_path(source, destination)
    
        i = 0
        j = 1
        node_i = path[i]
        node_j = path[j]
        
        current_distance = self.distance_between_nodes(node_i, node_j)
    
        while current_distance < distance_from_source:
            i += 1
            j += 1
            node_i = path[i]
            node_j = path[j]
        
            current_distance += self.distance_between_nodes(node_i, node_j)
        
        if current_distance == distance_from_source:
            return node_j
        else:
            distance_j = current_distance - distance_from_source
            return self.add_new_node_between_nodes(node_j, node_i, distance_j)

Funkcija **limb_length** izracunava duzina tzv. *spoljnje grane* za list **j** za datu matricu rastojanja **D** dimenzije **n** $\times$ **n**. Funkcija pored duzine spoljnje grane vraca i cvorove **i** i **k** na osnovu kojih je izracunata duzina spoljasnje grane za list **j**.

In [14]:
def limb_length(D, n, j):
    min_length = float('inf')
    min_i = None
    min_k = None

    # ======== STUDENTSKI KOD ======== #
    # u jbt sta bi formula za limb_length ? 
    # limb_length(j) = 0.5 * sum ( D[i][j] + D[k][j] - D[i][k]) da li bi ovo formula u
    for i in range(n):
        for k in range(n):
            if i != j and k != j:
                limb_length_j = 0.5 * ( D[i][j] + D[k][j] - D[i][k])
                
                if limb_length_j < min_length:
                    min_length = limb_length_j
                    min_i = i
                    min_k = k
                print(f'min_length {min_length}')
                print(f'limb_length_j {limb_length_j}')
                print(f'min_i {min_i}')
                print(f'min_k {min_k}')
                

    # retur
    # ================================ #
                    
    return (min_i, min_k, min_length)

Funkcija **additive_philogeny** konstruise filogenetsko stablo koje odgovara matrici rastojanja **D** dimenzije **n** $\times$ **n** primenom *algoritma aditivne filogenije*.

In [22]:
def additive_philogeny(D, n):
    # ======== STUDENTSKI KOD ======== #
    if n == 2:
        g = Graph({ 0 : [(1, D[0][1])], 
                 1 : [(0, D[0][1])]
               })
        print(g)
        return g

    j = n - 1 # biram list 

    (i, k, limb_length_j) = limb_length(D, n, j)
    print(f'ZA n={n}\ti: {i} k:{k} j:{j} limb_length_j:{limb_length_j}')

    # da li je to ovo j ? 
    # idmeo do n - 1 jer je oguljena 
    for k in range(n-1):
        D[k][j] -= limb_length_j
        D[j][k] -= limb_length_j 

    # D[0][3]  0 1 D[0][2]
    #              D[2][0]
    # D[3][0]
    # D[1][3]
        
    # ovde moram nesto da zapamtim 
    distance_i = D[i][j] # udaljenost lista j = n - 1 od lista i, u ogoljenoj matici D
    
    tree = additive_philogeny(D, n-1)

    # e sad ovde prespajanje tj formiranje grane 
    # cvor j umecemo izmedju i i k 
    # def add_node_on_path(self, source, destination, distance_from_source):
    m = tree.add_node_on_path(i, k, distance_i) 
    print(tree)
    
    #metod koji dodaje u graf suseda cvora node sa oznakom neighbor na udaljenosti distance
    # def add_neighbor(self, node, neighbor, distance):
    tree.add_neighbor(m, j, limb_length_j)
    print(tree)
    
    return tree
    # ================================ #

In [21]:
D = [[ 0, 13, 21, 22],
     [13,  0, 12, 13], 
     [21, 12,  0, 13], 
     [22, 13, 13,  0]]
n = 4

tree = additive_philogeny(D, n)
print(tree)

min_length 22.0
limb_length_j 22.0
min_i 0
min_k 0
min_length 11.0
limb_length_j 11.0
min_i 0
min_k 1
min_length 7.0
limb_length_j 7.0
min_i 0
min_k 2
min_length 7.0
limb_length_j 11.0
min_i 0
min_k 2
min_length 7.0
limb_length_j 13.0
min_i 0
min_k 2
min_length 7.0
limb_length_j 7.0
min_i 0
min_k 2
min_length 7.0
limb_length_j 7.0
min_i 0
min_k 2
min_length 7.0
limb_length_j 7.0
min_i 0
min_k 2
min_length 7.0
limb_length_j 13.0
min_i 0
min_k 2
ZA n=4	i: 0 k:2 j:3 limb_length_j:7.0
min_length 21.0
limb_length_j 21.0
min_i 0
min_k 0
min_length 10.0
limb_length_j 10.0
min_i 0
min_k 1
min_length 10.0
limb_length_j 10.0
min_i 0
min_k 1
min_length 10.0
limb_length_j 12.0
min_i 0
min_k 1
ZA n=3	i: 0 k:1 j:2 limb_length_j:10.0
{0: [(1, 13)], 1: [(0, 13)]}
{0: [('1+0', 11.0)], 1: [('1+0', 2.0)], '1+0': [(1, 2.0), (0, 11.0)]}
{0: [('1+0', 11.0)], 1: [('1+0', 2.0)], '1+0': [(1, 2.0), (0, 11.0), (2, 10.0)], 2: [('1+0', 10.0)]}
{0: [('1+0', 11.0)], 1: [('1+0', 2.0)], '1+0': [(1, 2.0), (0, 11.0), ('